<a href="https://colab.research.google.com/github/ananghw/project_simotandi/blob/main/WebScrapping_Simotandi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Prepare the library**

In [124]:
import requests
from bs4 import BeautifulSoup
%pip install tabula-py
%pip install JPype1
import pandas as pd
import tabula
from urllib.parse import urljoin
import os
import glob
from datetime import datetime
%pip install pandasql
from pandasql import sqldf

# **Build Dynamic URL**

In [125]:
base_url = 'https://simotandi.pertanian.go.id/datatabular.php'
page_url = 'https://simotandi.pertanian.go.id/datatabular_prov.php?no=57'

In [126]:
output_folder = 'hasil_scraping'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

kolom_header = [
    'No',
    'Provinsi/Kabupaten/Kota',
    'Bera',
    'Penggenangan',
    'Tanam (1-15 HST)',
    'Vegetatif 1 (16-30 HST)',
    'Vegetatif 2 (31-40 HST)',
    'Max. Vegetatif (41-54 HST)',
    'Generatif 1 (55-71 HST)',
    'Generatif 2 (72-110 HST)',
    'Panen',
    'Standing Crop',
    'Luas Baku Sawah (Ha)'
]

print(f"Mengambil halaman dari: {page_url}")
try:
    response = requests.get(page_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    pdf_links = soup.find_all('a', href=lambda href: href and '.pdf' in href)

    if not pdf_links:
        print("Tidak ada link PDF yang ditemukan di halaman ini.")
    else:
        total_pdfs = len(pdf_links)
        print(f"Ditemukan {total_pdfs} link PDF. Memulai proses ekstraksi...")

        for i, link_tag in enumerate(pdf_links):
            print(f"\n--- Memproses PDF {i+1}/{total_pdfs} ---")

            relative_link = link_tag['href']
            full_pdf_url = urljoin(base_url, relative_link)
            print(f"URL PDF: {full_pdf_url}")

            pdf_filename = os.path.basename(relative_link).replace('.pdf', '')
            output_excel_file = os.path.join(output_folder, f'tabel_dari_{pdf_filename}.xlsx')

            try:
                print("Mengekstrak tabel dari PDF...")
                tables = tabula.read_pdf(full_pdf_url, pages='all', lattice=True, multiple_tables=True, pandas_options={'header': None})

                if not tables:
                    print("Tidak ada tabel yang bisa diekstrak dari PDF ini.")
                else:
                    print(f"Berhasil! Ditemukan {len(tables)} tabel. Membersihkan dan menggabungkan...")

                    cleaned_tables_list = []

                    for j, df in enumerate(tables):
                        if df.empty:
                            continue

                        start_row_index = pd.to_numeric(df.iloc[:, 0], errors='coerce').first_valid_index()

                        if start_row_index is not None:
                            df_cleaned = df.loc[start_row_index:].reset_index(drop=True)

                            if len(df_cleaned.columns) == len(kolom_header):
                                df_cleaned.columns = kolom_header
                                # Masukkan tabel yang sudah bersih ke dalam list
                                cleaned_tables_list.append(df_cleaned)
                            else:
                                print(f"  Peringatan: Jumlah kolom di Tabel_{j+1} tidak cocok dengan header. Dilewati.")
                        else:
                            print(f"  Peringatan: Tidak ditemukan baris data numerik di Tabel_{j+1}. Dilewati.")

                    if cleaned_tables_list:
                        combined_df = pd.concat(cleaned_tables_list, ignore_index=True)

                        combined_df.to_excel(output_excel_file, index=False)
                        print(f"Semua tabel telah digabung dan disimpan ke: {output_excel_file}")
                    else:
                        print("Tidak ada tabel yang valid untuk digabungkan setelah proses pembersihan.")

            except Exception as e:
                print(f"GAGAL memproses PDF ini. Error: {e}")

except requests.exceptions.RequestException as e:
    print(f"Gagal mengakses URL halaman web: {e}")

print("\n\n--- Proses Selesai ---")

Mengambil halaman dari: https://simotandi.pertanian.go.id/datatabular_prov.php?no=57
Ditemukan 34 link PDF. Memulai proses ekstraksi...

--- Memproses PDF 1/34 ---
URL PDF: https://simotandi.pertanian.go.id/pdf/571104b571b7a13c5a05b7d5694c1e8af306.pdf
Mengekstrak tabel dari PDF...
Berhasil! Ditemukan 7 tabel. Membersihkan dan menggabungkan...
Semua tabel telah digabung dan disimpan ke: hasil_scraping/tabel_dari_571104b571b7a13c5a05b7d5694c1e8af306.xlsx

--- Memproses PDF 2/34 ---
URL PDF: https://simotandi.pertanian.go.id/pdf/571227552720a7e5efdc4adea1338f805220.pdf
Mengekstrak tabel dari PDF...
Berhasil! Ditemukan 10 tabel. Membersihkan dan menggabungkan...
Semua tabel telah digabung dan disimpan ke: hasil_scraping/tabel_dari_571227552720a7e5efdc4adea1338f805220.xlsx

--- Memproses PDF 3/34 ---
URL PDF: https://simotandi.pertanian.go.id/pdf/571356fd3896aaec780f1275d599f5fc416f.pdf
Mengekstrak tabel dari PDF...
Berhasil! Ditemukan 5 tabel. Membersihkan dan menggabungkan...
Semua tabel 

In [127]:
#CHECKING THE SAMPLE XLSX
excel_file_path = "/content/hasil_scraping/tabel_dari_571104b571b7a13c5a05b7d5694c1e8af306.xlsx"
df_sample = pd.read_excel(excel_file_path)
display(df_sample.head())

print(f"Number of rows: {df_sample.shape[0]}")
print(f"Number of columns: {df_sample.shape[1]}")

,No,Provinsi/Kabupaten/Kota,Bera,Penggenangan,Tanam (1-15 HST),Vegetatif 1 (16-30 HST),Vegetatif 2 (31-40 HST),Max. Vegetatif (41-54 HST),Generatif 1 (55-71 HST),Generatif 2 (72-110 HST),Panen,Standing Crop,Luas Baku Sawah (Ha)
0,1,Aceh,74.988,16.344,18.000,15.951,12.391,18.769,17.068,13.864,26.228,96.043,214.729
1,2,Aceh Barat,3.643,1.192,932,672,329,746,785,477,1.348,3.941,10.192
2,3,Arongan Lambalek,375,109,28,23,22,138,215,50,217,476,1.188
3,4,Bubon,220,83,73,88,56,59,45,12,85,333,733
4,5,Johan Pahlawan,98,24,28,20,14,30,14,20,111,126,363


Number of rows: 313
Number of columns: 13


In [128]:
#COMBINE ALL THE DATA EXTRACTED


print("\nMenggabungkan seluruh file Excel menjadi satu file...")

# Cari semua file Excel di folder output
excel_files = glob.glob(os.path.join(output_folder, 'tabel_dari_*.xlsx'))

combined_all_df = []

for file in excel_files:
    try:
        df = pd.read_excel(file)
        df['Source File'] = os.path.basename(file)  # Tambahkan kolom sumber
        combined_all_df.append(df)
    except Exception as e:
        print(f"Gagal membaca {file}: {e}")

if combined_all_df:
    final_combined_df = pd.concat(combined_all_df, ignore_index=True)
    combined_output_file = os.path.join(output_folder, 'gabungan_semua_tabel.xlsx')
    final_combined_df.to_excel(combined_output_file, index=False)
    print(f"Berhasil menggabungkan semua file ke: {combined_output_file}")
else:
    print("Tidak ada file yang berhasil digabung.")

#CHECKING THE SAMPLE XLSX
excel_file_path = "/content/hasil_scraping/gabungan_semua_tabel.xlsx"
df_sample = pd.read_excel(excel_file_path)
display(df_sample.head())

print(f"Number of rows: {df_sample.shape[0]}")
print(f"Number of columns: {df_sample.shape[1]}")


Menggabungkan seluruh file Excel menjadi satu file...
Berhasil menggabungkan semua file ke: hasil_scraping/gabungan_semua_tabel.xlsx


,No,Provinsi/Kabupaten/Kota,Bera,Penggenangan,Tanam (1-15 HST),Vegetatif 1 (16-30 HST),Vegetatif 2 (31-40 HST),Max. Vegetatif (41-54 HST),Generatif 1 (55-71 HST),Generatif 2 (72-110 HST),Panen,Standing Crop,Luas Baku Sawah (Ha),Source File
0,1,Bali,19.049,4.935,5.233,6.967,6.951,4.749,4.843,4.547,13.611,33.290,72.006,tabel_dari_575114db44f6943261e99aad97b1e546e63...
1,2,Badung,2.678,645,673,720,576,575,660,867,1.722,4.071,9.216,tabel_dari_575114db44f6943261e99aad97b1e546e63...
2,3,Abiansemal,804,183,185,225,128,111,156,251,724,1.056,2.800,tabel_dari_575114db44f6943261e99aad97b1e546e63...
3,4,Kuta,2,-,-,-,-,-,-,-,1,-,3,tabel_dari_575114db44f6943261e99aad97b1e546e63...
4,5,Kuta Selatan,-,-,-,-,-,-,-,-,-,-,-,tabel_dari_575114db44f6943261e99aad97b1e546e63...


Number of rows: 7541
Number of columns: 14


In [129]:
#EXTRACT DATA PROVINSI
query = "SELECT * FROM df_sample WHERE No = '1'"
data_provinsi = sqldf(query)

print(f"Number of rows: {data_provinsi.shape[0]}")
print(f"Number of columns: {data_provinsi.shape[1]}")

data_provinsi.head()

Number of rows: 34
Number of columns: 14


,No,Provinsi/Kabupaten/Kota,Bera,Penggenangan,Tanam (1-15 HST),Vegetatif 1 (16-30 HST),Vegetatif 2 (31-40 HST),Max. Vegetatif (41-54 HST),Generatif 1 (55-71 HST),Generatif 2 (72-110 HST),Panen,Standing Crop,Luas Baku Sawah (Ha),Source File
0,1,Bali,19.049,4.935,5.233,6.967,6.951,4.749,4.843,4.547,13.611,33.290,72.006,tabel_dari_575114db44f6943261e99aad97b1e546e63...
1,1,Kalimantan Selatan,"101,046","13,163","14,740","14,820","17,958","40,971","28,092","15,186","44,657","131,767","292,992",tabel_dari_5763567d57691a30da3afdbcddcea910c20...
2,1,Sulawesi Utara,13.372,3.622,3.362,2.900,2.852,5.754,5.051,3.437,6.119,23.356,46.878,tabel_dari_5771bda87e786a087c911f53e7b015cb400...
3,1,Jawa Tengah,269.089,62.600,74.142,101.146,96.039,133.033,126.166,51.736,132.228,582.262,1.061.636,tabel_dari_5733626edf55583206e795c97210fa0c3a7...
4,1,Jawa Barat,297.601,50.775,61.931,90.249,97.112,108.186,69.095,42.460,109.888,469.033,937.352,tabel_dari_57327e852490f5963f684da96982b31b4a5...


In [130]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [132]:
# Replace 'path/to/your/csv_file.csv' with the actual path to your CSV file in Google Drive
csv_file_path = '/content/drive/MyDrive/data simotandi - dictionary kota_kabupaten.csv'

try:
    df_drive = pd.read_csv(csv_file_path)
    display(df_drive)
except FileNotFoundError:
    print(f"Error: The file '{csv_file_path}' was not found. Please check the path and try again.")
except Exception as e:
    print(f"An error occurred: {e}")


,PROVINSI,KOTA/KABUPATEN
0,ACEH,ACEH BARAT
1,ACEH,ACEH BARAT DAYA
2,ACEH,ACEH BESAR
3,ACEH,ACEH JAYA
4,ACEH,ACEH SELATAN
...,...,...
509,SUMATERA UTARA,TAPANULI SELATAN
510,SUMATERA UTARA,TAPANULI TENGAH
511,SUMATERA UTARA,TAPANULI UTARA
512,SUMATERA UTARA,TEBING TINGGI


In [134]:
#EXTRACT DATA KOTA/KABUPATEN
query = "SELECT * FROM df_sample WHERE No != '1'"
data_kabupaten = sqldf(query)

print(f"Number of rows: {data_kabupaten.shape[0]}")
print(f"Number of columns: {data_kabupaten.shape[1]}")

data_kabupaten.head()

Number of rows: 7507
Number of columns: 14


,No,Provinsi/Kabupaten/Kota,Bera,Penggenangan,Tanam (1-15 HST),Vegetatif 1 (16-30 HST),Vegetatif 2 (31-40 HST),Max. Vegetatif (41-54 HST),Generatif 1 (55-71 HST),Generatif 2 (72-110 HST),Panen,Standing Crop,Luas Baku Sawah (Ha),Source File
0,2,Badung,2.678,645,673,720,576,575,660,867,1.722,4.071,9.216,tabel_dari_575114db44f6943261e99aad97b1e546e63...
1,3,Abiansemal,804,183,185,225,128,111,156,251,724,1.056,2.800,tabel_dari_575114db44f6943261e99aad97b1e546e63...
2,4,Kuta,2,-,-,-,-,-,-,-,1,-,3,tabel_dari_575114db44f6943261e99aad97b1e546e63...
3,5,Kuta Selatan,-,-,-,-,-,-,-,-,-,-,-,tabel_dari_575114db44f6943261e99aad97b1e546e63...
4,6,Kuta Utara,318,72,88,70,51,98,82,95,111,484,996,tabel_dari_575114db44f6943261e99aad97b1e546e63...


In [135]:
# Convert the 'Provinsi/Kabupaten/Kota' column to uppercase in data_kabupaten
data_kabupaten['Provinsi/Kabupaten/Kota'] = data_kabupaten['Provinsi/Kabupaten/Kota'].str.upper()

# Perform a left merge to keep all rows from data_kabupaten and include the matching columns from df_drive
merged_df = data_kabupaten.merge(df_drive[['PROVINSI', 'KOTA/KABUPATEN']],
                                  left_on='Provinsi/Kabupaten/Kota',
                                  right_on='KOTA/KABUPATEN',
                                  how='left',
                                  indicator=True)

# Create the 'label_kabupaten' column
merged_df['label_kabupaten'] = merged_df['_merge'].apply(lambda x: 'yes' if x == 'both' else 'no')

# Rename the KOTA/KABUPATEN column from df_drive and drop the merge indicator
merged_df = merged_df.rename(columns={'KOTA/KABUPATEN': 'Matched_KOTA/KABUPATEN'}).drop(columns=['_merge'])

# Display the first few rows with the new columns
display(merged_df.head())

# Display the number of rows where label_kabupaten is 'yes'
print(f"Number of rows labeled 'yes': {merged_df[merged_df['label_kabupaten'] == 'yes'].shape[0]}")

,No,Provinsi/Kabupaten/Kota,Bera,Penggenangan,Tanam (1-15 HST),Vegetatif 1 (16-30 HST),Vegetatif 2 (31-40 HST),Max. Vegetatif (41-54 HST),Generatif 1 (55-71 HST),Generatif 2 (72-110 HST),Panen,Standing Crop,Luas Baku Sawah (Ha),Source File,PROVINSI,Matched_KOTA/KABUPATEN,label_kabupaten
0,2,BADUNG,2.678,645,673,720,576,575,660,867,1.722,4.071,9.216,tabel_dari_575114db44f6943261e99aad97b1e546e63...,BALI,BADUNG,yes
1,3,ABIANSEMAL,804,183,185,225,128,111,156,251,724,1.056,2.800,tabel_dari_575114db44f6943261e99aad97b1e546e63...,NaN,NaN,no
2,4,KUTA,2,-,-,-,-,-,-,-,1,-,3,tabel_dari_575114db44f6943261e99aad97b1e546e63...,NaN,NaN,no
3,5,KUTA SELATAN,-,-,-,-,-,-,-,-,-,-,-,tabel_dari_575114db44f6943261e99aad97b1e546e63...,NaN,NaN,no
4,6,KUTA UTARA,318,72,88,70,51,98,82,95,111,484,996,tabel_dari_575114db44f6943261e99aad97b1e546e63...,NaN,NaN,no


Number of rows labeled 'yes': 675


In [136]:
# Filter the DataFrame to only include rows where 'label_kabupaten' is 'yes'
filtered_df = merged_df[merged_df['label_kabupaten'] == 'yes']

# Display the first few rows of the filtered DataFrame
display(filtered_df.head())

# Display the number of rows in the filtered DataFrame
print(f"Number of rows where label_kabupaten is 'yes': {filtered_df.shape[0]}")

,No,Provinsi/Kabupaten/Kota,Bera,Penggenangan,Tanam (1-15 HST),Vegetatif 1 (16-30 HST),Vegetatif 2 (31-40 HST),Max. Vegetatif (41-54 HST),Generatif 1 (55-71 HST),Generatif 2 (72-110 HST),Panen,Standing Crop,Luas Baku Sawah (Ha),Source File,PROVINSI,Matched_KOTA/KABUPATEN,label_kabupaten
0,2,BADUNG,2.678,645,673,720,576,575,660,867,1.722,4.071,9.216,tabel_dari_575114db44f6943261e99aad97b1e546e63...,BALI,BADUNG,yes
7,9,BANGLI,653,112,73,214,261,138,122,125,486,933,2.222,tabel_dari_575114db44f6943261e99aad97b1e546e63...,BALI,BANGLI,yes
8,10,BANGLI,169,23,17,59,78,23,35,29,157,241,606,tabel_dari_575114db44f6943261e99aad97b1e546e63...,BALI,BANGLI,yes
12,14,BULELENG,2.129,607,1.030,1.071,1.311,639,736,394,898,5.181,8.966,tabel_dari_575114db44f6943261e99aad97b1e546e63...,BALI,BULELENG,yes
13,15,BANJAR,175,46,69,73,58,46,78,34,124,358,718,tabel_dari_575114db44f6943261e99aad97b1e546e63...,KALIMANTAN SELATAN,BANJAR,yes


Number of rows where label_kabupaten is 'yes': 675


In [138]:
#CHECKIN SAMPLE DATA
filtered_df_provinsi = filtered_df[filtered_df['PROVINSI'] == 'DI YOGYAKARTA']
# Display the number of rows where provinsi is 'x'
print(f"Kota Kabupaten dengan Provinsi: {filtered_df_provinsi[filtered_df_provinsi['PROVINSI'] == 'DI YOGYAKARTA'].shape[0]}")


filtered_df_provinsi = filtered_df_provinsi.sort_values(by='Provinsi/Kabupaten/Kota')
filtered_df_provinsi

Kota Kabupaten dengan Provinsi: 7


,No,Provinsi/Kabupaten/Kota,Bera,Penggenangan,Tanam (1-15 HST),Vegetatif 1 (16-30 HST),Vegetatif 2 (31-40 HST),Max. Vegetatif (41-54 HST),Generatif 1 (55-71 HST),Generatif 2 (72-110 HST),Panen,Standing Crop,Luas Baku Sawah (Ha),Source File,PROVINSI,Matched_KOTA/KABUPATEN,label_kabupaten
4019,2,BANTUL,3.581,1.291,1.031,1.264,1.095,1.891,2.289,615,1.805,8.185,15.123,tabel_dari_5734a46a1d85395cad2b007f57c280ebb31...,DI YOGYAKARTA,BANTUL,yes
4022,5,BANTUL,109,62,82,129,145,129,163,44,41,692,929,tabel_dari_5734a46a1d85395cad2b007f57c280ebb31...,DI YOGYAKARTA,BANTUL,yes
4037,20,GUNUNG KIDUL,15.024,1.324,1.913,2.910,730,787,1.175,1.634,6.816,9.149,32.456,tabel_dari_5734a46a1d85395cad2b007f57c280ebb31...,DI YOGYAKARTA,GUNUNG KIDUL,yes
4056,39,KOTA YOGYAKARTA,14,2,3,6,3,3,5,2,10,22,48,tabel_dari_5734a46a1d85395cad2b007f57c280ebb31...,DI YOGYAKARTA,KOTA YOGYAKARTA,yes
4071,54,KULON PROGO,3.445,713,955,756,675,1.310,1.202,405,1.466,5.303,11.165,tabel_dari_5734a46a1d85395cad2b007f57c280ebb31...,DI YOGYAKARTA,KULON PROGO,yes
4084,67,SLEMAN,5.363,987,1.285,1.780,1.894,2.530,1.587,567,2.112,9.643,18.563,tabel_dari_5734a46a1d85395cad2b007f57c280ebb31...,DI YOGYAKARTA,SLEMAN,yes
4099,82,SLEMAN,305,52,79,192,182,306,122,21,44,902,1.351,tabel_dari_5734a46a1d85395cad2b007f57c280ebb31...,DI YOGYAKARTA,SLEMAN,yes


In [139]:
df = filtered_df.copy()

# Treat 'Bera' as thousands-formatted string (e.g., '3.581' = 3581)
df['Bera_num'] = pd.to_numeric(df['Bera'].str.replace('.', '', regex=False), errors='coerce')

# Sort as intended
df_sorted = df.sort_values(by=['Provinsi/Kabupaten/Kota', 'Bera_num', 'No'], ascending=[True, False, True])

# Drop duplicates by 'Provinsi/Kabupaten/Kota'
result_df = df_sorted.drop_duplicates(subset=['Provinsi/Kabupaten/Kota'], keep='first')

# Clean up
result_df = result_df.drop(columns='Bera_num')
print(result_df)

       No Provinsi/Kabupaten/Kota    Bera Penggenangan Tanam (1-15 HST)  \
6046    2              ACEH BARAT   3.643        1.192              932   
6059   15         ACEH BARAT DAYA   2.313          550            1.390   
6069   25              ACEH BESAR   8.791        1.939            1.923   
6093   49               ACEH JAYA   2.746          523              426   
6103   59            ACEH SELATAN   2.423          526              568   
...   ...                     ...     ...          ...              ...   
4337  237               WAY KANAN   4.083          632            1.692   
980   569                WONOGIRI  13.748        1.573            1.738   
1006  595                WONOSOBO   4.323          777              961   
5802  397                YAHUKIMO       -            -                -   
5854  449                  YALIMO       -            -                -   

     Vegetatif 1 (16-30 HST) Vegetatif 2 (31-40 HST)  \
6046                     672               

In [143]:
result_df

,No,Provinsi/Kabupaten/Kota,Bera,Penggenangan,Tanam (1-15 HST),Vegetatif 1 (16-30 HST),Vegetatif 2 (31-40 HST),Max. Vegetatif (41-54 HST),Generatif 1 (55-71 HST),Generatif 2 (72-110 HST),Panen,Standing Crop,Luas Baku Sawah (Ha),Source File,PROVINSI,Matched_KOTA/KABUPATEN,label_kabupaten
6046,2,ACEH BARAT,3.643,1.192,932,672,329,746,785,477,1.348,3.941,10.192,tabel_dari_571104b571b7a13c5a05b7d5694c1e8af30...,ACEH,ACEH BARAT,yes
6059,15,ACEH BARAT DAYA,2.313,550,1.390,811,461,780,628,504,867,4.574,8.341,tabel_dari_571104b571b7a13c5a05b7d5694c1e8af30...,ACEH,ACEH BARAT DAYA,yes
6069,25,ACEH BESAR,8.791,1.939,1.923,1.346,1.652,3.717,2.776,1.545,2.047,12.959,25.827,tabel_dari_571104b571b7a13c5a05b7d5694c1e8af30...,ACEH,ACEH BESAR,yes
6093,49,ACEH JAYA,2.746,523,426,400,533,826,1.123,1.007,1.272,4.315,8.888,tabel_dari_571104b571b7a13c5a05b7d5694c1e8af30...,ACEH,ACEH JAYA,yes
6103,59,ACEH SELATAN,2.423,526,568,383,298,788,772,563,1.119,3.372,7.491,tabel_dari_571104b571b7a13c5a05b7d5694c1e8af30...,ACEH,ACEH SELATAN,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4337,237,WAY KANAN,4.083,632,1.692,1.125,1.120,1.495,1.395,1.000,1.848,7.827,14.584,tabel_dari_571824951d71ff815d5f973db2ae1087a09...,LAMPUNG,WAY KANAN,yes
980,569,WONOGIRI,13.748,1.573,1.738,3.618,4.712,6.292,3.494,2.265,6.627,22.119,44.538,tabel_dari_5733626edf55583206e795c97210fa0c3a7...,JAWA TENGAH,WONOGIRI,yes
1006,595,WONOSOBO,4.323,777,961,1.109,597,881,1.138,369,1.784,5.055,12.182,tabel_dari_5733626edf55583206e795c97210fa0c3a7...,JAWA TENGAH,WONOSOBO,yes
5802,397,YAHUKIMO,-,-,-,-,-,-,-,-,-,-,-,tabel_dari_579474122982237e4240a6f30b74ee42f96...,PAPUA,YAHUKIMO,yes


In [144]:
# Check the data types of each column in result_df
print(result_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 513 entries, 6046 to 5854
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   No                          513 non-null    int64 
 1   Provinsi/Kabupaten/Kota     513 non-null    object
 2   Bera                        513 non-null    object
 3   Penggenangan                513 non-null    object
 4   Tanam (1-15 HST)            513 non-null    object
 5   Vegetatif 1 (16-30 HST)     513 non-null    object
 6   Vegetatif 2 (31-40 HST)     513 non-null    object
 7   Max. Vegetatif (41-54 HST)  513 non-null    object
 8   Generatif 1 (55-71 HST)     513 non-null    object
 9   Generatif 2 (72-110 HST)    513 non-null    object
 10  Panen                       513 non-null    object
 11  Standing Crop               513 non-null    object
 12  Luas Baku Sawah (Ha)        513 non-null    object
 13  Source File                 513 non-null    object


In [158]:
columns_to_convert = [
    'Bera',
    'Penggenangan',
    'Tanam (1-15 HST)',
    'Vegetatif 1 (16-30 HST)',
    'Vegetatif 2 (31-40 HST)',
    'Max. Vegetatif (41-54 HST)',
    'Generatif 1 (55-71 HST)',
    'Generatif 2 (72-110 HST)',
    'Panen',
    'Standing Crop',
    'Luas Baku Sawah (Ha)'
]

# Show original and converted values side by side
for col in columns_to_convert:
    result_df[f"{col}_raw"] = filtered_df[col]  # store original
    result_df[col] = pd.to_numeric(
        filtered_df[col].astype(str).str.replace('.', '', regex=False), errors='coerce'
    )

# Check what's happening
#print(result_df[[f"{col}_raw", col] for col in columns_to_convert].head(10))


# Check the data types again to confirm the conversion
#print(result_df.info())

# Display the first few rows to see the converted values
#display(result_df.head())

print(result_df)


       No Provinsi/Kabupaten/Kota     Bera  Penggenangan  Tanam (1-15 HST)  \
6046    2              ACEH BARAT   3643.0        1192.0             932.0   
6059   15         ACEH BARAT DAYA   2313.0         550.0            1390.0   
6069   25              ACEH BESAR   8791.0        1939.0            1923.0   
6093   49               ACEH JAYA   2746.0         523.0             426.0   
6103   59            ACEH SELATAN   2423.0         526.0             568.0   
...   ...                     ...      ...           ...               ...   
4337  237               WAY KANAN   4083.0         632.0            1692.0   
980   569                WONOGIRI  13748.0        1573.0            1738.0   
1006  595                WONOSOBO   4323.0         777.0             961.0   
5802  397                YAHUKIMO      NaN           NaN               NaN   
5854  449                  YALIMO      NaN           NaN               NaN   

      Vegetatif 1 (16-30 HST)  Vegetatif 2 (31-40 HST)  \
6046 

In [160]:
resulted = result_df[result_df['PROVINSI'] == 'DI YOGYAKARTA']
# Display the number of rows where provinsi is 'x'
print(f"Kota Kabupaten dengan Provinsi: {result_df[result_df['PROVINSI'] == 'DI YOGYAKARTA'].shape[0]}")


resulted = resulted.sort_values(by='Provinsi/Kabupaten/Kota')
print(resulted)

Kota Kabupaten dengan Provinsi: 5
      No Provinsi/Kabupaten/Kota     Bera  Penggenangan  Tanam (1-15 HST)  \
4019   2                  BANTUL   3581.0        1291.0            1031.0   
4037  20            GUNUNG KIDUL  15024.0        1324.0            1913.0   
4056  39         KOTA YOGYAKARTA     14.0           2.0               3.0   
4071  54             KULON PROGO   3445.0         713.0             955.0   
4084  67                  SLEMAN   5363.0         987.0            1285.0   

      Vegetatif 1 (16-30 HST)  Vegetatif 2 (31-40 HST)  \
4019                   1264.0                   1095.0   
4037                   2910.0                    730.0   
4056                      6.0                      3.0   
4071                    756.0                    675.0   
4084                   1780.0                   1894.0   

      Max. Vegetatif (41-54 HST)  Generatif 1 (55-71 HST)  \
4019                      1891.0                   2289.0   
4037                       787.0

In [166]:
resulf_df = result_df
result_df.info()

# Identify columns to exclude (those ending with '_raw')
columns_to_exclude = [col for col in result_df.columns if col.endswith('_raw')]

# Create a new DataFrame excluding the identified columns
cleaned_result_df = result_df.drop(columns=columns_to_exclude)

# Display the first few rows of the cleaned DataFrame
display(cleaned_result_df.head())

# Print the info of the cleaned DataFrame to confirm column removal and data types
print(cleaned_result_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 513 entries, 6046 to 5854
Data columns (total 28 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   No                              513 non-null    int64  
 1   Provinsi/Kabupaten/Kota         513 non-null    object 
 2   Bera                            446 non-null    float64
 3   Penggenangan                    451 non-null    float64
 4   Tanam (1-15 HST)                452 non-null    float64
 5   Vegetatif 1 (16-30 HST)         453 non-null    float64
 6   Vegetatif 2 (31-40 HST)         453 non-null    float64
 7   Max. Vegetatif (41-54 HST)      448 non-null    float64
 8   Generatif 1 (55-71 HST)         452 non-null    float64
 9   Generatif 2 (72-110 HST)        452 non-null    float64
 10  Panen                           448 non-null    float64
 11  Standing Crop                   446 non-null    float64
 12  Luas Baku Sawah (Ha)            427 n

,No,Provinsi/Kabupaten/Kota,Bera,Penggenangan,Tanam (1-15 HST),Vegetatif 1 (16-30 HST),Vegetatif 2 (31-40 HST),Max. Vegetatif (41-54 HST),Generatif 1 (55-71 HST),Generatif 2 (72-110 HST),Panen,Standing Crop,Luas Baku Sawah (Ha),Source File,PROVINSI,Matched_KOTA/KABUPATEN,label_kabupaten
6046,2,ACEH BARAT,3643.0,1192.0,932.0,672.0,329.0,746.0,785.0,477.0,1348.0,3941.0,10192.0,tabel_dari_571104b571b7a13c5a05b7d5694c1e8af30...,ACEH,ACEH BARAT,yes
6059,15,ACEH BARAT DAYA,2313.0,550.0,1390.0,811.0,461.0,780.0,628.0,504.0,867.0,4574.0,8341.0,tabel_dari_571104b571b7a13c5a05b7d5694c1e8af30...,ACEH,ACEH BARAT DAYA,yes
6069,25,ACEH BESAR,8791.0,1939.0,1923.0,1346.0,1652.0,3717.0,2776.0,1545.0,2047.0,12959.0,25827.0,tabel_dari_571104b571b7a13c5a05b7d5694c1e8af30...,ACEH,ACEH BESAR,yes
6093,49,ACEH JAYA,2746.0,523.0,426.0,400.0,533.0,826.0,1123.0,1007.0,1272.0,4315.0,8888.0,tabel_dari_571104b571b7a13c5a05b7d5694c1e8af30...,ACEH,ACEH JAYA,yes
6103,59,ACEH SELATAN,2423.0,526.0,568.0,383.0,298.0,788.0,772.0,563.0,1119.0,3372.0,7491.0,tabel_dari_571104b571b7a13c5a05b7d5694c1e8af30...,ACEH,ACEH SELATAN,yes


<class 'pandas.core.frame.DataFrame'>
Index: 513 entries, 6046 to 5854
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   No                          513 non-null    int64  
 1   Provinsi/Kabupaten/Kota     513 non-null    object 
 2   Bera                        446 non-null    float64
 3   Penggenangan                451 non-null    float64
 4   Tanam (1-15 HST)            452 non-null    float64
 5   Vegetatif 1 (16-30 HST)     453 non-null    float64
 6   Vegetatif 2 (31-40 HST)     453 non-null    float64
 7   Max. Vegetatif (41-54 HST)  448 non-null    float64
 8   Generatif 1 (55-71 HST)     452 non-null    float64
 9   Generatif 2 (72-110 HST)    452 non-null    float64
 10  Panen                       448 non-null    float64
 11  Standing Crop               446 non-null    float64
 12  Luas Baku Sawah (Ha)        427 non-null    float64
 13  Source File                 513 non-

In [174]:
cleaned_result_df
cleaned_result = cleaned_result_df[cleaned_result_df['PROVINSI'] == 'DI YOGYAKARTA']
cleaned_result

,No,Provinsi/Kabupaten/Kota,Bera,Penggenangan,Tanam (1-15 HST),Vegetatif 1 (16-30 HST),Vegetatif 2 (31-40 HST),Max. Vegetatif (41-54 HST),Generatif 1 (55-71 HST),Generatif 2 (72-110 HST),Panen,Standing Crop,Luas Baku Sawah (Ha),Source File,PROVINSI,Matched_KOTA/KABUPATEN,label_kabupaten
4019,2,BANTUL,3581.0,1291.0,1031.0,1264.0,1095.0,1891.0,2289.0,615.0,1805.0,8185.0,15123.0,tabel_dari_5734a46a1d85395cad2b007f57c280ebb31...,DI YOGYAKARTA,BANTUL,yes
4037,20,GUNUNG KIDUL,15024.0,1324.0,1913.0,2910.0,730.0,787.0,1175.0,1634.0,6816.0,9149.0,32456.0,tabel_dari_5734a46a1d85395cad2b007f57c280ebb31...,DI YOGYAKARTA,GUNUNG KIDUL,yes
4056,39,KOTA YOGYAKARTA,14.0,2.0,3.0,6.0,3.0,3.0,5.0,2.0,10.0,22.0,48.0,tabel_dari_5734a46a1d85395cad2b007f57c280ebb31...,DI YOGYAKARTA,KOTA YOGYAKARTA,yes
4071,54,KULON PROGO,3445.0,713.0,955.0,756.0,675.0,1310.0,1202.0,405.0,1466.0,5303.0,11165.0,tabel_dari_5734a46a1d85395cad2b007f57c280ebb31...,DI YOGYAKARTA,KULON PROGO,yes
4084,67,SLEMAN,5363.0,987.0,1285.0,1780.0,1894.0,2530.0,1587.0,567.0,2112.0,9643.0,18563.0,tabel_dari_5734a46a1d85395cad2b007f57c280ebb31...,DI YOGYAKARTA,SLEMAN,yes


In [176]:
# prompt: save the gabungan_semua_tabel into google sheet

# Install gspread and google-auth
!pip install --upgrade gspread google-auth

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
import pandas as pd
import numpy as np # Import numpy to handle NaN and inf

# Authenticate and create a client
creds, _ = default()
gc = gspread.authorize(creds)

# Name of the sheet you want to save to
spreadsheet_name = 'data simotandi'
worksheet_name = 'data_kota_kabupaten' # Or any other sheet name

try:
    # Try to open the spreadsheet by name
    spreadsheet = gc.open(spreadsheet_name)
except gspread.SpreadsheetNotFound:
    # If not found, create a new one
    print(f"Spreadsheet '{spreadsheet_name}' not found. Creating a new one...")
    spreadsheet = gc.create(spreadsheet_name)
    print(f"Spreadsheet '{spreadsheet_name}' created.")

# Select the worksheet. If it doesn't exist, create it (gspread v5+ handles this)
try:
    worksheet = spreadsheet.worksheet(worksheet_name)
except gspread.WorksheetNotFound:
    print(f"Worksheet '{worksheet_name}' not found. Creating a new one...")
    worksheet = spreadsheet.add_worksheet(title=worksheet_name, rows="100", cols="20") # Adjust rows/cols as needed
    print(f"Worksheet '{worksheet_name}' created.")


# Assuming 'cleaned_result_df' is the DataFrame you want to save
# If it's not already defined, you need to ensure the preceding code
# successfully creates the 'cleaned_result_df' DataFrame.

if 'cleaned_result_df' in locals(): # Check for cleaned_result_df instead of final_combined_df
    print(f"Saving data to Google Sheet '{spreadsheet_name}' in worksheet '{worksheet_name}'...")

    # Replace NaN and infinite values with None for JSON compliance
    cleaned_data_for_upload = cleaned_result_df.replace({np.nan: None, np.inf: None, -np.inf: None})


    # Convert DataFrame to a list of lists (including header)
    data_to_upload = [cleaned_data_for_upload.columns.values.tolist()] + cleaned_data_for_upload.values.tolist()

    # Upload the data
    worksheet.update(values=data_to_upload)

    print("Data successfully uploaded to Google Sheet.")
else:
    print("Error: 'cleaned_result_df' DataFrame was not created. Please ensure the preceding code ran successfully.")

Saving data to Google Sheet 'data simotandi' in worksheet 'data_kota_kabupaten'...
Data successfully uploaded to Google Sheet.


After mounting your Google Drive, you can specify the path to your CSV file and read it into a pandas DataFrame. Remember to replace `'path/to/your/csv_file.csv'` with the actual path to your file in Google Drive.